# 模型构建、训练与评估

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# 为了在plt中正确显示中文
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

## 1 加载训练/测试数据

In [3]:
# 加载数据
fund_data = pd.read_csv('../data/processed/基金processed.csv')

client_train_x = pd.read_csv('../data/processed/客户train_x.csv')
rating_train = pd.read_csv('../data/processed/评分train.csv')

client_test_x = pd.read_csv('../data/processed/客户test_x.csv')
rating_test = pd.read_csv('../data/processed/评分test.csv')

## 2 基础模型的选择与构建、以及评估

基础模型, 我们首先考虑非深度学习模型, 比如:
1. 协同过滤: pros:只需要客户/基金的交互数据即可进行运算, 速度很快; cons:忽略了大量特征信息, 且它所依赖的评分数据是我们人为构建的.
2. content-based推荐: pros:可以根据客户对基金的喜好进行类似基金的推荐, 或对拥有类似特征的客户推荐基金, 计算成本低; cons: 没有对客户和基金的特征进行交叉计算, 可能会忽略他们之间的相关性
3. 因子分解机: pros:能够模拟特征之间的所有交互，适用于数据稀疏的情况，同时能够集成多种类型的数据; cons:训练过程相对复杂和计算密集.

由于我们有充足的客户/基金数据, 且特征数量较多, 我们可以选择能充分利用特征的模型, 所以我们将:
1. 以content-based方法开始, 实现最快最直观的推荐
2. 进一步迁移到其他模型, 比如充分利用特征交叉的因子分解机FM
3. 如果计算资源条件允许, 且对于精度要求很高, 那么考虑使用深度学习模型, 比如DeepFM

### 2.1 Content-Based 推荐

1. 此方法不需要区分训练/测试集, 直接通过向量之间的距离进行计算, 这里以测试集为例. 
2. 对于最佳的推荐选项, 我选择了 相似度*评分
3. 对于此方法的评估方法, 可以根据推荐结果的top-n选项进行accuracy和recall rate的评估

#### 2.1a 基于客户相似度的推荐

In [4]:
# 构建客户索引映射
client_index_mapping = {client_id: i for i, client_id in enumerate(client_test_x['客户编号'].unique())}
fund_index_mapping = {i: fund_id for i, fund_id in enumerate(rating_test.columns)}

num_clients = len(client_index_mapping)
num_funds = rating_test.shape[1] - 1

# 构建评分矩阵
rating_matrix = rating_test.drop('客户编号', axis=1).values

# 计算客户之间的余弦相似度
client_similarities = cosine_similarity(client_test_x.drop('客户编号', axis=1))

# 计算推荐分数
recommendation_scores = np.dot(client_similarities, rating_matrix)

# 选择每个客户的顶部基金推荐
top_n = 8
top_recommendations = np.argsort(recommendation_scores, axis=1)[:, -top_n:]

# 转换推荐结果为DataFrame
client_ids = client_test_x['客户编号'].values
fund_ids = [fund_index_mapping[i] for i in range(num_funds)]
recommendation_df = pd.DataFrame(top_recommendations, index=client_ids, columns=[f'Top_{i+1}' for i in range(top_n)])
recommendation_df = recommendation_df.applymap(lambda x: fund_ids[x])

print(recommendation_df.head())


       Top_1  Top_2  Top_3  Top_4  Top_5  Top_6  Top_7  Top_8
C1936  J0144  J0122  J0022  J0018  J0118  J0155  J0076  J0063
C6495  J0145  J0029  J0009  J0185  J0054  J0111  J0070  J0183
C1721  J0151  J0009  J0136  J0097  J0110  J0135  J0167  J0172
C9121  J0075  J0039  J0022  J0154  J0169  J0064  J0193  J0016
C0361  J0168  J0126  J0144  J0131  J0107  J0065  J0027  J0071


#### 2.1b 基于基金相似度的推荐

In [9]:
# 计算基金之间的余弦相似度
fund_similarities = cosine_similarity(fund_data.drop('基金代码', axis=1))

# 创建基金索引映射
fund_index_mapping = {fund_id: i for i, fund_id in enumerate(fund_data['基金代码'].unique())}

# 创建评分矩阵，初始值为0
num_clients = client_test_x['客户编号'].nunique()
num_funds = len(fund_index_mapping)
rating_matrix = rating_test.drop('客户编号', axis=1).values

# 使用相似度和评分计算推荐分数
recommendation_scores = np.dot(rating_matrix, fund_similarities)

# 选择每个客户的顶部基金推荐
top_n = 8
top_recommendations = np.argsort(recommendation_scores, axis=1)[:, -top_n:]

# 转换推荐结果为DataFrame
client_ids = list(client_index_mapping.keys())
fund_ids = list(fund_index_mapping.keys())
recommendation_df = pd.DataFrame(top_recommendations, index=client_ids, columns=[f'Top_{i+1}' for i in range(top_n)])
recommendation_df = recommendation_df.applymap(lambda x: fund_ids[x])

print(recommendation_df.head())


       Top_1  Top_2  Top_3  Top_4  Top_5  Top_6  Top_7  Top_8
C1936  J0099  J0165  J0195  J0081  J0082  J0004  J0012  J0096
C6495  J0068  J0069  J0070  J0071  J0072  J0073  J0063  J0200
C1721  J0153  J0174  J0161  J0169  J0136  J0100  J0098  J0173
C9121  J0068  J0069  J0070  J0071  J0072  J0073  J0063  J0200
C0361  J0068  J0069  J0070  J0071  J0072  J0073  J0063  J0200


### 2.2 因子分解机FM

In [ ]:
from fastFM import als
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, recall_score, precision_score

In [ ]:
# 处理客户和基金数据, 使之结合成大的稀疏矩阵, 作为训练/测试集的X
def prepare_fm_data(client_data, fund_data):

    # 创建客户和基金的 one-hot 编码矩阵
    client_onehot = csr_matrix(pd.get_dummies(client_data['客户编号'], sparse=True))
    fund_onehot = csr_matrix(pd.get_dummies(fund_data['基金代码'], sparse=True))

    # 将数值特征转换为稀疏矩阵
    client_features = csr_matrix(client_data.drop(['客户编号'], axis=1).values)
    fund_features = csr_matrix(fund_data.drop(['基金代码'], axis=1).values)

    # 合并所有特征为一个大的稀疏矩阵
    X = hstack([client_onehot, fund_onehot, client_features, fund_features])

    return X

In [ ]:
# 构建FM模型并训练

# 客户和基金的索引映射需要在训练集上创建，并在测试集上使用相同的映射
client_index_mapping = {client_id: i for i, client_id in enumerate(client_train_x['客户编号'].unique())}
fund_index_mapping = {fund_id: i for i, fund_id in enumerate(fund_data['基金代码'].unique())}

# 处理训练数据
X_train = prepare_fm_data(client_train_x, fund_data)
y_train = rating_train

# 处理测试数据
X_test = prepare_fm_data(client_test_x, fund_data)
y_test = rating_test

# 使用 FM 模型进行训练和预测
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=2, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)

### 2.3 DeepFM

In [ ]:
from deepctr.models import DeepFM

In [ ]:
# 处理训练数据
X_train = prepare_fm_data(client_train_x, fund_data)
y_train = rating_train

# 处理测试数据
X_test = prepare_fm_data(client_test_x, fund_data)
y_test = rating_test

# 取出所有的列名
all_feature_columns = X_train.columns.tolist()

# 为DeepFM创建特征列
linear_feature_columns = all_feature_columns
dnn_feature_columns = all_feature_columns

# 创建DeepFM模型
task_type = 'regression'
deepfm_model = DeepFM(linear_feature_columns, dnn_feature_columns, task=task_type)

# 编译模型
deepfm_model.compile(optimizer='adam', loss='mean_squared_error' if task_type == 'regression' else 'binary_crossentropy')

# 训练模型
deepfm_model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2)


## 3 模型的评估: 离线指标与线上指标

### 3.1 离线指标

除了从数学角度选取了基于连续值的RMSE、MAE, 我们还选取了具有实际意义的基于离散值指标:
1. 准确率（Accuracy）：衡量模型正确推荐用户感兴趣的基金的比例。这样可以通过减少错误的推荐，确保用户看到的基金是他们可能真正感兴趣的, 提高用户体验
2. 召回率（Recall）：衡量模型成功推荐用户感兴趣的基金的比例，考虑了模型漏掉的推荐。高召回率表示模型能够捕捉到更多用户的兴趣，确保不错过可能的好机会

In [ ]:
# 预测测试集上的评分
model = fm
y_pred = model.predict(X_test)

# 计算均方根误差（RMSE）和平均绝对误差（MAE）（不进行二值化处理）
rmse = np.sqrt(mean_squared_error(rating_test, y_pred))
mae = mean_absolute_error(rating_test, y_pred)

print("RMSE:", rmse)
print("MAE:", mae)


# 将预测评分二值化，只要评分不为0，就记为1
y_pred_binary = (y_pred > 0).astype(int)

# 计算准确率、召回率和精确度
accuracy = accuracy_score(rating_test > 0, y_pred_binary)
recall = recall_score(rating_test > 0, y_pred_binary, average='micro')
precision = precision_score(rating_test > 0, y_pred_binary, average='micro')

print("Accuracy (1/0):", accuracy)
print("Recall (1/0):", recall)
print("Precision (1/0):", precision)

### 3.2 线上指标
如果有机会将模型上线, 那么可以选取:
1. 点击率（CTR）：衡量用户点击推荐内容的比例。CTR衡量了客户对推荐内容的兴趣程度
2. 转化率（Conversion Rate）：我们的目标是促使客户购买基金，所以转化率是一个关键指标，它衡量了成功推荐的比例